<h2> In the name of God</h2>
<h3>Signal and systems Project phase2</h3>
<h4>Nima Samadi 97102011</h4>

In [3]:
import pydub
import numpy as np
import sounddevice as sd
from scipy.io.wavfile import read, write
from Music import Music
import tkinter as tk
from tkinter import filedialog
from scipy.signal import butter, lfilter, freqz
from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg, NavigationToolbar2Tk)
from matplotlib.backend_bases import key_press_handler
from matplotlib.figure import Figure
from PIL import ImageTk, Image
import os
import threading
import time
from matplotlib import pyplot as plt
import matplotlib.font_manager as mfm
from matplotlib.figure import Figure
import subprocess

<h3 style="color:red">Main Gui</h3>
<h4> This section contain Main GUI class with sub GUI classes:</h4>

In [6]:
class MainGUI:
    def __init__(self):
        self.root = tk.Tk()
        self.root.geometry("500x200")
        self.root.title("Audio Processing GUI")
    
    def show(self):
        Xoffset = 0
        Yoffset = 0
    
        img = ImageTk.PhotoImage(Image.open("1.jpg"))
        panel = tk.Label(self.root, image = img)
        panel.image = img
        panel.pack(side = "bottom", fill = "both", expand = "yes")

        tk.Label(self.root, text="Run one of the following GUI: ").place(x=Xoffset+20, y=Yoffset+30)
        tk.Button(self.root, text="Frequncy Filter GUI", command=self.FrequencyFilter).place(x=Xoffset+30, y=Yoffset+60)
        tk.Button(self.root, text="Chorus GUI", command=self.Chorus).place(x=Xoffset+30, y=Yoffset+90)
        tk.Button(self.root, text="3D Effect GUI", command=self.Effect).place(x=Xoffset+30, y=Yoffset+120)
        tk.Button(self.root, text="Sound Decomposition GUI", command=self.SoundDecomp).place(x=Xoffset+30, y=Yoffset+150)
        
        self.root.mainloop()
        
    def FrequencyFilter(self):
        window1 = FrequencyFilterGUI()
        window1.show()
            
    def Chorus(self):
        window2 = ChorusGUI()
        window2.show()
    
    def Effect(self):
        window3 = EffectGUI()
        window3.show()
    
    def SoundDecomp(self):
        window4 = noteDetection()
        window4.show()
        
class FrequencyFilterGUI:
    def __init__(self):
        self.top = tk.Tk()
   
    def show(self):
        self.top.geometry("950x570")
        self.top.title("Frequncy Filter GUI")
        Xoffset = 10;
        Yoffset = 70;
        tk.Label(self.top, text="Enter time period [s]: ").place(x=10+Xoffset, y=30+Yoffset)
        self.initial = tk.Entry(self.top, width=7)
        self.initial.place(x=127+Xoffset, y=30+Yoffset)
        tk.Label(self.top, text=":").place(x=174+Xoffset, y=30+Yoffset)
        self.final = tk.Entry(self.top, width=7)
        self.final.place(x=185+Xoffset, y=30+Yoffset)
                
        tk.Label(self.top, text="Enter frequncy Band [Hz]: ").place(x=10+Xoffset, y=60+Yoffset)
        self.initialF = tk.Entry(self.top, width=7)
        self.initialF.place(x=152+Xoffset, y=60+Yoffset)
        tk.Label(self.top, text=":").place(x=195+Xoffset, y=60+Yoffset)
        self.finalF = tk.Entry(self.top, width=7)
        self.finalF.place(x=210+Xoffset, y=60+Yoffset)

        tk.Label(self.top, text="Enter frequncy filter order: ").place(x=10+Xoffset, y=90+Yoffset)
        self.order = tk.Entry(self.top, width=7)
        self.order.place(x=160+Xoffset, y=90+Yoffset)

        tk.Button(self.top, text="Browse a music", command=self.BrowseMusic).place(x=40+Xoffset, y=150+Yoffset)
        
        tk.Button(self.top, text="Play original music", command=self.PlayOriginalMusic).place(x=40+Xoffset, y=180+Yoffset)
    
        tk.Button(self.top, text="Run & play filtered music", command=self.PlayMusic).place(x=40+Xoffset, y=210+Yoffset)
        
        tk.Button(self.top, text="Save filtered music", command=self.saveMusic).place(x=40+Xoffset, y=240+Yoffset)
        tk.Button(self.top, text="Stop", command=sd.stop).place(x=155+Xoffset, y=240+Yoffset)
        
        self.top.mainloop()
    
    def BrowseMusic(self):
        self.filename = filedialog.askopenfilename(initialdir='/', title="Select a music", filetype=[("mp3", "*.mp3")])
        self.fs1, self.data = Music.read_mp3(self.filename, True)
        self.ti = self.initial.get()
        self.tf = self.final.get()
        
    def PlayOriginalMusic(self):
        self.ti = self.initial.get()
        self.tf = self.final.get()
        Y = self.data[int(float(self.ti)*44100): int(float(self.tf)*44100), 1];
        Y /= np.max(np.abs(Y))
        sd.play(Y, self.fs1)

    def saveMusic(self):
        self.filesave = filedialog.asksaveasfile(initialdir='/', title="Save as ", filetype=[("Audio file", "*.mp3"), ("All file", "*.*")], defaultextension=".mp3")
        Music.write_mp3(self.filesave.name, self.fs1, self.filteredSignal, normalized=True) 
           
    def PlayMusic(self):
        plotFrame = tk.Frame(self.top)
        plotFrame.place(x=300, y=40)
        fig = Figure(figsize=(6, 5), dpi=100)
        y = self.data[int(float(self.ti)*44100): int(float(self.tf)*44100), 1]
        y = y / np.max(np.abs(y))
        
        self.Fi = self.initialF.get()
        self.Ff = self.initialF.get()     
        Order = self.order.get();
        lowcut = self.initialF.get()
        highcut = self.finalF.get()
        
        axes1 = fig.add_subplot(211)
        t = np.linspace(float(self.ti), float(self.tf), len(y))
        axes1.plot(t, y)

        axes1.set_title("Original music")
        axes1.set_xlabel("Time [s]")
        axes1.set_ylabel("F(t)")
        axes1.grid(True)
        
        axes2 = fig.add_subplot(212)
        self.filteredSignal = self.butter_bandpass_filter(y, float(lowcut), float(highcut), int(self.fs1), int(Order))
        self.filteredSignal /= np.max(np.abs(self.filteredSignal))
        axes2.plot(t, self.filteredSignal)
        axes2.set_title("Filtered music")
        axes2.set_xlabel("Time [s]")
        axes2.set_ylabel("F(t)")
        axes2.grid(True)

        fig.set_tight_layout('tight')
        
        canvas = FigureCanvasTkAgg(fig, master=plotFrame)
        toolbar = NavigationToolbar2Tk(canvas, plotFrame)
        toolbar.update()
        canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)
        canvas.draw()
        canvas.get_tk_widget().pack(side=tk.BOTTOM, fill=tk.BOTH, expand=1)
        sd.play(self.filteredSignal, int(self.fs1))
        
    def butter_bandpass_filter(self, data, lowcut, highcut, fs, order=5):
        b, a = self.butter_bandpass(lowcut, highcut, fs, order=order)
        y = lfilter(b, a, data)
        return y

    def butter_bandpass(self, lowcut, highcut, fs, order=5):
        nyq = 0.5 * fs
        low = lowcut / nyq
        high = highcut / nyq
        b, a = butter(order, [low, high], btype='band')
        return b, a

    
class ChorusGUI():
    def __init__(self):
        self.top = tk.Tk()
    
    def show(self):
        self.top.geometry("500x400")
        self.top.title("Chorus GUI")
        Xoffset = 10;
        Yoffset = 0;
        
        tk.Label(self.top, text="Number of chorus (k): ").place(x=15+Xoffset, y=50+Yoffset)
        self.K = tk.Entry(self.top, width=7)
        self.K.place(x=140+Xoffset, y=50+Yoffset)
        
        tk.Label(self.top, text="Value of " + 'Δ' + 't [ms]: ').place(x=15+Xoffset, y=75+Yoffset)
        self.T = tk.Entry(self.top, width=7)
        self.T.place(x=110+Xoffset, y=75+Yoffset)
        
        tk.Label(self.top, text="Time period [s]: ").place(x=15+Xoffset, y=100+Yoffset)
        self.initial = tk.Entry(self.top, width=7)
        self.initial.place(x=105+Xoffset, y=100+Yoffset)
        tk.Label(self.top, text=":").place(x=150+Xoffset, y=100+Yoffset)
        self.final = tk.Entry(self.top, width=7)
        self.final.place(x=160+Xoffset, y=100+Yoffset)

        tk.Button(self.top, text="Select a music", command=self.BrowseMusic).place(x=40+Xoffset, y=240+Yoffset)
        tk.Button(self.top, text="Play original music", command=self.playOriginalMusic).place(x=130+Xoffset, y=240+Yoffset)
        tk.Button(self.top, text="Apply chorus & play", command=self.applyChorus).place(x=40+Xoffset, y=270+Yoffset)
        tk.Button(self.top, text="Stop", command=sd.stop).place(x=165+Xoffset, y=270+Yoffset)
        tk.Button(self.top, text="Save music", command=self.saveMusic).place(x=40+Xoffset, y=300+Yoffset)
        
        self.top.mainloop()
    
    def BrowseMusic(self):
        self.filename = filedialog.askopenfilename(initialdir='/', title="Select a music", filetype=[("mp3", "*.mp3")])
        self.fs1, self.data = Music.read_mp3(self.filename, True)
    
    def saveMusic(self):
        self.filesave = filedialog.asksaveasfile(initialdir='/', title="Save as ", filetype=[("Audio file", "*.mp3"), ("All file", "*.*")], defaultextension=".mp3")
        Music.write_mp3(self.filesave.name, self.fs1, self.YC, normalized=True) 
    
    def playOriginalMusic(self):
        self.ti = self.initial.get()
        self.tf = self.final.get()
        Y = self.data[int(float(self.ti)*44100): int(float(self.tf)*44100), 1];
        Y /= np.max(np.abs(Y))
        sd.play(Y, self.fs1)
        
    def applyChorus(self):
        self.ti = self.initial.get()
        self.tf = self.final.get()
        self.Kval = self.K.get()
        self.Tval = self.T.get()
        mus = 0
        for i in range(int(self.Kval)):
            mus += self.data[int((float(self.ti)*44100))+int(float(self.Tval)*i*44.1): int((float(self.tf)*44100))+int(float(self.Tval)*i*44.1), 1];
        mus /= np.max(np.abs(mus))
        self.YC = mus
        sd.play(self.YC, self.fs1)
        
class EffectGUI:
    def __init__(self):
        self.top = tk.Tk()
        self.top.geometry("700x400") 
        self.top.title("3D Effect GUI")
        self.val = 0
        
    def show(self):        
        Xoffset = 50;
        Yoffset = 0;        
        
        self.slider = tk.Scale(self.top, from_=-1, to=1, orient=tk.HORIZONTAL, resolution=0.1, command=self.getNew)
        self.slider.place(x=270+Xoffset, y=30+Yoffset)  
        
        tk.Label(self.top, text="Enter time period [s]: ").place(x=10+Xoffset, y=40+Yoffset)
        self.initial = tk.Entry(self.top, width=7)
        self.initial.place(x=127+Xoffset, y=40+Yoffset)
        tk.Label(self.top, text=":").place(x=175+Xoffset, y=40+Yoffset)
        self.final = tk.Entry(self.top, width=7)
        self.final.place(x=185+Xoffset, y=40+Yoffset)
        
        tk.Label(self.top, text='L').place(x=270+Xoffset, y = 10+Yoffset)
        tk.Label(self.top, text='R').place(x=360+Xoffset, y = 10+Yoffset)
        
        tk.Button(self.top, text="Play", command=self.Play).place(x=40+Xoffset, y=110+Yoffset)
        tk.Button(self.top, text="Stop", command=sd.stop).place(x=80+Xoffset, y=110+Yoffset)
        tk.Button(self.top, text="Select a music", command=self.BrowseMusic).place(x=40+Xoffset, y=140+Yoffset)
        tk.Button(self.top, text="Save music", command=self.saveMusic).place(x=130+Xoffset, y=140+Yoffset)
        
        self.top.mainloop()
    
    def BrowseMusic(self):
        self.filename = filedialog.askopenfilename(initialdir='/', title="Select a music", filetype=[("mp3", "*.mp3")])
        self.fs1, self.data = Music.read_mp3(self.filename, True)
    
    def saveMusic(self):
        self.filesave = filedialog.asksaveasfile(initialdir='/', title="Save as ", filetype=[("Audio file", "*.mp3"), ("All file", "*.*")], defaultextension=".mp3")
        Music.write_mp3(self.filesave.name, self.fs1, self.Y, normalized=True) 

    def getNew(self, event):
        self.val = self.slider.get()
        
    def Play(self):
        self.ti = self.initial.get()
        self.tf = self.final.get()
        l = int(float(self.ti)*44100)
        r = int(float(self.tf)*44100)
        self.Y = np.zeros((r-l, 2))
        if self.val >= 0:
            self.Y[:, 0] = np.multiply(self.data[l:r, 1], 1-self.val)
            self.Y[:, 1] = np.multiply(self.data[l:r, 0], 1)
        else:
            self.Y[:, 0] = np.multiply(self.data[l:r, 1], 1)
            self.Y[:, 1] = np.multiply(self.data[l:r, 0], 1+self.val)
        
        sd.play(self.Y, self.fs1)
        
class noteDetection:
    def __init__(self):
        self.top = tk.Tk()
        self.top.geometry("400x500")
        self.top.title("Note Detection")
    
    def show(self):
        Xoffset = 0
        Yoffset = 0
        tk.Label(self.top, text="Enter time period [s]: ").place(x=10+Xoffset, y=30+Yoffset)
        self.initial = tk.Entry(self.top, width=7)
        self.initial.place(x=127+Xoffset, y=30+Yoffset)
        tk.Label(self.top, text=":").place(x=174+Xoffset, y=30+Yoffset)
        self.final = tk.Entry(self.top, width=7)
        self.final.place(x=185+Xoffset, y=30+Yoffset)
        
        tk.Button(self.top, text="Browse a music", command=self.BrowseMusic).place(x=40+Xoffset, y=150+Yoffset)
        tk.Button(self.top, text="Play original music", command=self.PlayOriginalMusic).place(x=40+Xoffset, y=180+Yoffset)
        tk.Button(self.top, text="Extract Notes", command=self.ExtractNote).place(x=40+Xoffset, y=210+Yoffset)    
        tk.Button(self.top, text="Save Note Sheet", command=self.NoteSheet).place(x=40+Xoffset, y=240+Yoffset)    
        tk.Button(self.top, text="Play Notes", command=self.PlayNotes).place(x=130+Xoffset, y=210+Yoffset)    
        tk.Button(self.top, text="Stop", command=sd.stop).place(x=160+Xoffset, y=180+Yoffset)
        
        self.top.mainloop()
    
    def ExtractNote(self):
        self.ti = self.initial.get()
        self.tf = self.final.get()
        INITIAL = int(float(self.ti) * self.fs1)
        FINAL = int(float(self.tf) * self.fs1)
        y = self.data[INITIAL:FINAL, 1]
        mus = Filter(y, self.fs1)
        y = mus.butter_bandpass_filter(200, 2000)
        t = np.linspace(INITIAL, FINAL, len(y))
        t /= self.fs1
        # 30 ms windows:
        RMSValue = np.zeros(len(y)*100//(self.fs1*3))
        windowSize = int(30 * 10 ** (-3) * self.fs1)
        # RMS Values:
        for i in range(len(RMSValue)):
            y = self.data[INITIAL+windowSize*i: INITIAL+windowSize*(i+1), 1]
            RMS = (np.mean(y**2))**(0.5)
            RMSValue[i] = RMS
        # compare RMS to threshold
        threshold = np.mean(RMSValue)
        for i in range(len(RMSValue)):
            if RMSValue[i] >= 0.8*threshold:
                RMSValue[i] = 1
            else:
                RMSValue[i] = 0
        notes = []
        for i in range(len(RMSValue)-1):
            y = self.data[INITIAL+windowSize*i: INITIAL+windowSize*(i+1), 1]
            if RMSValue[i]:
                pNote = self.noteDetect(y, self.fs1)
                notes.append(pNote[0][0])
            elif i != (len(y) - 1) and RMSValue[i] == 0 and RMSValue[i+1] == 1:
                pNote = self.noteDetect(y, self.fs1)
                nNote = self.noteDetect(self.data[INITIAL+windowSize*(i+1): INITIAL+windowSize*(i+2), 1], self.fs1)
                for val in pNote:
                    if np.abs(val[0]-nNote[0][0]) <= 1:
                        notes.append(val[0])
            elif RMSValue[i] == 0 and RMSValue[i-1] == 1 and i != 0:
                pNote = self.noteDetect(y, self.fs1)        
                bNote = self.noteDetect(self.data[INITIAL+windowSize*(i-1): INITIAL+windowSize*(i), 1], self.fs1)
                for val in pNote:
                    if np.abs(val[0]-bNote[0][0]) <= 1:
                        notes.append(val[0])
            else:
                notes.append(0)

            noteNum = []
        for val in notes:
            k = 3
            if val != 0:
                if val >= 220 and val < 440:
                    k = 3
                    base = 2 ** (k - 1) * 55
                    noteNum.append([int(12*np.log2(val / base)), k])
                elif val >= 440 and val < 880:
                    k = 4
                    base = 2 ** (k - 1) * 55
                    noteNum.append([int(12*np.log2(val / base)), k])
                elif val >= 880 and val < 1760:
                    k = 5
                    base = 2 ** (k - 1) * 55
                    noteNum.append([int(12*np.log2(val / base)), k])
            else:
                noteNum.append([-1, -1])
        for i in range(len(noteNum)):
            if noteNum[i][0] == 11:
                noteNum[i] = [0, noteNum[i][1] + 1]
            elif noteNum[i][0] == 9:
                noteNum[i] = [10, noteNum[i][1]]
            elif noteNum[i][0] == 1 or noteNum[i][0] == 4 or noteNum[i][0] == 6 or noteNum[i][0] == 9:
                noteNum[i] = [noteNum[i][0] + 1, noteNum[i][1]]

        noteDuration = []
        duration = 30
        for i in range(len(noteNum)):
            if i != len(noteNum) - 1: 
                if noteNum[i][0] == noteNum[i+1][0] and noteNum[i][1] == noteNum[i+1][1] :
                    duration += 30
                else:
                    noteDuration.append([noteNum[i][0], noteNum[i][1], duration])
                    duration = 30
            else:
                noteDuration.append([noteNum[i][0], noteNum[i][1], duration])

        # delete short notes and 0:
        noteDur = []            
        for i in range(len(noteDuration)):
            if noteDuration[i][0] != -1 and noteDuration[i][2] != 30:
                noteDur.append(noteDuration[i])
        self.noteDuration = []
        # refind note duration
        duration = 0
        for i in range(len(noteDur)):
            duration += noteDur[i][2]
            if i != len(noteDur) - 1: 
                if noteDur[i][0] == noteDur[i+1][0] and noteDur[i][1] == noteDur[i+1][1] :
                    pass
                else:
                    self.noteDuration.append([noteDur[i][0], noteDur[i][1], duration])
                    duration = 0
            elif i == len(noteDur) - 1:
                self.noteDuration.append([noteDur[i][0], noteDur[i][1], duration])

        notes_duration = np.array([3200, 1600, 800, 400, 200, 100])*0.5

        for i in range(len(self.noteDuration)):
            if self.noteDuration[i][2] > notes_duration[1] and self.noteDuration[i][2] < notes_duration[0]:
                self.noteDuration[i][2] = 0
            elif self.noteDuration[i][2] > notes_duration[2] and self.noteDuration[i][2] < notes_duration[1]:
                self.noteDuration[i][2] = 1
            elif self.noteDuration[i][2] > notes_duration[3] and self.noteDuration[i][2] < notes_duration[2]:
                self.noteDuration[i][2] = 2
            elif self.noteDuration[i][2] > notes_duration[4] and self.noteDuration[i][2] < notes_duration[3]:
                self.noteDuration[i][2] = 3
            elif self.noteDuration[i][2] > notes_duration[5] and self.noteDuration[i][2] < notes_duration[4]:
                self.noteDuration[i][2] = 4
            elif self.noteDuration[i][2] < notes_duration[5]:
                self.noteDuration[i][2] = 5
    
    def NoteSheet(self):
        note = self.noteDuration
        sign = ['𝅝', '𝅗𝅥', '𝅘𝅥', '♪', '𝅘𝅥𝅯', '𝅘𝅥𝅰']
        font_path = 'C:\Windows\Fonts\seguisym.ttf'
        prop = mfm.FontProperties(fname=font_path)  
        plotFrame = tk.Frame(self.top)
        plotFrame.place(x=300, y=40)
        fig = Figure(figsize=(16, 20), dpi=100)
        axes1 = fig.add_subplot(111)
        xP = 0
        yP = 81
        yB = {3:9.5, 4:4.5, 5:-0.5, 6:-5.5,}
        yS = {0:0, 2:1, 3:1.5, 5:2.5, 7:3, 8:4, 10:4.5}
        for i in range(10):
                for j in range(5):
                    axes1.plot(np.linspace(0, 60, 100), np.ones(100)*(15*i + 1.5*j), color='black')

        for i in range(len(note)):
            if xP == 60:
                yP -= 15
            if yP < 0:
                break
            xP = xP % 60
            axes1.text(xP, yP-yB[note[i][1]]+yS[note[i][0]], s=sign[note[i][2]], fontproperties=prop, fontsize=48)
            xP += 2

        axes1.text(25, 95, "Music 1", fontsize=32)
        axes1.set_ylim([-7, 86])
        axes1.axis(False)
        filesave = filedialog.asksaveasfile(initialdir='/', title="Save as ", filetype=[("PDF file", "*.pdf")], defaultextension=".pdf")
        fig.savefig(filesave.name)
        filesave.close()
        subprocess.Popen([filesave.name],shell=True)
        
    def noteDetect(self, data, fs):
        y = data
        f = np.fft.fft(y)
        f = np.abs(f)
        p = np.linspace(0, fs, y.shape[0])
        notes = []
        for _ in range(2):
            index = np.argmax(f[:70])
            a = p[index]
            b = f[index]
            notes.append([a, b])
            f[index] = 0
        return notes
    
    def BrowseMusic(self):
        self.filename = filedialog.askopenfilename(initialdir='/', title="Select a music", filetype=[("mp3", "*.mp3")])
        self.fs1, self.data = Music.read_mp3(self.filename, True)
        self.ti = self.initial.get()
        self.tf = self.final.get()
    
    def PlayOriginalMusic(self):
        self.ti = self.initial.get()
        self.tf = self.final.get()
        Y = self.data[int(float(self.ti)*44100): int(float(self.tf)*44100), 1];
        Y /= np.max(np.abs(Y))
        sd.play(Y, self.fs1)        
    
    def PlayNotes(self):
        def sin_wave(f, n, fs):
            x = np.linspace(0, 2*np.pi, n)
            xp = np.linspace(0, -1*(n*30/fs), n)
            y = np.sin(x*f*(n/fs))*np.exp(xp)
            z = np.zeros([n, 2])
            z[:, 0] = y
            z[:, 1] = y
            return z

        def put_note(note_id, octave, dur, fs):
            y = sin_wave(notes_base[note_id]*2**octave, int(notes_duration[dur]*(fs/1000)), fs)
            return y

        def get_music(music_notes, fs): 
            m = []
            for item in music_notes:
                y = put_note(item[0], item[1], item[2], fs)
                m.append(y)
            m = np.concatenate(m, 0)
            return m
        
        notes_duration = np.array([3200, 1600, 800, 400, 200, 100])*0.5
        notes_base = 2**(np.arange(12)/12)*27.5
        music = self.noteDuration
        y = get_music(music, self.fs1)
        sd.play(y, self.fs1)

class Filter: 
    def __init__(self, data, fs):
        self.data = data
        self.fs = fs

    def butter_bandpass(self, lowcut, highcut, fs, order=5):
        nyq = 0.5 * fs
        low = lowcut / nyq
        high = highcut / nyq
        b, a = butter(order, [low, high], btype='band')
        return b, a

    def butter_bandpass_filter(self, lowcut, highcut, order=5):
        b, a = self.butter_bandpass(lowcut, highcut, self.fs, order)
        y = lfilter(b, a, self.data)
        return y
          
        
Window = MainGUI()
Window.show()

<h4>Sub classes seprately:</h4>

In [60]:
class FrequncyFilterGUI:
    def __init__(self):
        self.top = tk.Tk()
   
    def show(self):
        self.top.geometry("950x570")
        self.top.title("Frequncy Filter GUI")
        Xoffset = 10;
        Yoffset = 70;
        timePeriod = tk.Label(self.top, text="Enter time period [s]: ")
        timePeriod.place(x=10+Xoffset, y=30+Yoffset)
        self.initial = tk.Entry(self.top, width=7)
        self.initial.place(x=127+Xoffset, y=30+Yoffset)
        char1 = tk.Label(self.top, text=":")
        char1.place(x=174+Xoffset, y=30+Yoffset)
        self.final = tk.Entry(self.top, width=7)
        self.final.place(x=185+Xoffset, y=30+Yoffset)
                
        frequencyBand = tk.Label(self.top, text="Enter frequncy Band [Hz]: ")
        frequencyBand.place(x=10+Xoffset, y=60+Yoffset)
        self.initialF = tk.Entry(self.top, width=7)
        self.initialF.place(x=152+Xoffset, y=60+Yoffset)
        char1 = tk.Label(self.top, text=":")
        char1.place(x=195+Xoffset, y=60+Yoffset)
        self.finalF = tk.Entry(self.top, width=7)
        self.finalF.place(x=210+Xoffset, y=60+Yoffset)

        OrderL = tk.Label(self.top, text="Enter frequncy filter order: ")
        OrderL.place(x=10+Xoffset, y=90+Yoffset)
        self.order = tk.Entry(self.top, width=7)
        self.order.place(x=160+Xoffset, y=90+Yoffset)

        button1 = tk.Button(self.top, text="Browse a music", command=self.BrowseMusic)
        button1.place(x=40+Xoffset, y=150+Yoffset)
        
        button3= tk.Button(self.top, text="Play original music", command=self.PlayOriginalMusic)
        button3.place(x=40+Xoffset, y=180+Yoffset)
        
        button2 = tk.Button(self.top, text="Run & play filtered music", command=self.PlayMusic)
        button2.place(x=40+Xoffset, y=210+Yoffset)

        self.top.mainloop()
    
    def BrowseMusic(self):
        self.filename = filedialog.askopenfilename(initialdir='/', title="Select a music", filetype=[("mp3", "*.mp3")])
        # music signal and time values:
        self.fs1, self.data = Music.read_mp3(self.filename, True)
        self.ti = self.initial.get()
        self.tf = self.final.get()
          
            
        
    def PlayOriginalMusic(self):
        self.ti = self.initial.get()
        self.tf = self.final.get()
        Y = self.data[int(float(self.ti)*44100): int(float(self.tf)*44100), 1];
        Y /= np.max(np.abs(Y))
        sd.play(Y, self.fs1)
#         sd.wait()
                
    def PlayMusic(self):
        # plot
        plotFrame = tk.Frame(self.top)
        plotFrame.place(x=300, y=40)
        fig = Figure(figsize=(6, 5), dpi=100)
        # filter:
        y = self.data[int(float(self.ti)*44100): int(float(self.tf)*44100), 1]
        y = y / np.max(np.abs(y))
        
        self.Fi = self.initialF.get()
        self.Ff = self.initialF.get()     
        Order = self.order.get();
        lowcut = self.initialF.get()
        highcut = self.finalF.get()
        
        axes1 = fig.add_subplot(211)
        t = np.linspace(float(self.ti), float(self.tf), len(y))
        axes1.plot(t, y)

        axes1.set_title("Original music")
        axes1.set_xlabel("Time [s]")
        axes1.set_ylabel("F(t)")
        axes1.grid(True)
        
        axes2 = fig.add_subplot(212)
        filteredSignal = self.butter_bandpass_filter(y, float(lowcut), float(highcut), int(self.fs1), int(Order))
        filteredSignal /= np.max(np.abs(filteredSignal))
        axes2.plot(t, filteredSignal)
        axes2.set_title("Filtered music")
        axes2.set_xlabel("Time [s]")
        axes2.set_ylabel("F(t)")
        axes2.grid(True)

        fig.set_tight_layout('tight')
        
        canvas = FigureCanvasTkAgg(fig, master=plotFrame)
        toolbar = NavigationToolbar2Tk(canvas, plotFrame)
        toolbar.update()
        canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)
        canvas.draw()
        canvas.get_tk_widget().pack(side=tk.BOTTOM, fill=tk.BOTH, expand=1)
        #play music        
        sd.play(filteredSignal, int(self.fs1))
#         sd.wait()
        
    def butter_bandpass_filter(self, data, lowcut, highcut, fs, order=5):
        b, a = self.butter_bandpass(lowcut, highcut, fs, order=order)
        y = lfilter(b, a, data)
        return y

    def butter_bandpass(self, lowcut, highcut, fs, order=5):
        nyq = 0.5 * fs
        low = lowcut / nyq
        high = highcut / nyq
        b, a = butter(order, [low, high], btype='band')
        return b, a
        
        
window1 = FrequncyFilterGUI()
window1.show()



In [36]:
class ChorusGUI():
    def __init__(self):
        self.top = tk.Tk()
    
    def show(self):
        self.top.geometry("500x400")
        self.top.title("Chorus GUI")
        Xoffset = 10;
        Yoffset = 0;
        
        numberK = tk.Label(self.top, text="Number of chorus (k): ")
        numberK.place(x=15+Xoffset, y=50+Yoffset)
        self.K = tk.Entry(self.top, width=7)
        self.K.place(x=140+Xoffset, y=50+Yoffset)
        
        delT = tk.Label(self.top, text="Value of " + 'Δ' + 't [ms]: ')
        delT.place(x=15+Xoffset, y=75+Yoffset)
        self.T = tk.Entry(self.top, width=7)
        self.T.place(x=110+Xoffset, y=75+Yoffset)
        
        timePeriod = tk.Label(self.top, text="Time period [s]: ")
        timePeriod.place(x=15+Xoffset, y=100+Yoffset)
        self.initial = tk.Entry(self.top, width=7)
        self.initial.place(x=105+Xoffset, y=100+Yoffset)
        char1 = tk.Label(self.top, text=":")
        char1.place(x=150+Xoffset, y=100+Yoffset)
        self.final = tk.Entry(self.top, width=7)
        self.final.place(x=160+Xoffset, y=100+Yoffset)

        button1 = tk.Button(self.top, text="Select a music", command=self.BrowseMusic)
        button1.place(x=40+Xoffset, y=240+Yoffset)
        
        button2 = tk.Button(self.top, text="Play original music", command=self.playOriginalMusic)
        button2.place(x=130+Xoffset, y=240+Yoffset)
        
        
        button3 = tk.Button(self.top, text="Apply chorus & play", command=self.applyChorus)
        button3.place(x=40+Xoffset, y=270+Yoffset)
        
        button4 = tk.Button(self.top, text="Save music", command=self.saveMusic)
        button4.place(x=40+Xoffset, y=300+Yoffset)

        
        self.top.mainloop()
    
    def BrowseMusic(self):
        self.filename = filedialog.askopenfilename(initialdir='/', title="Select a music", filetype=[("mp3", "*.mp3")])
        self.fs1, self.data = Music.read_mp3(self.filename, True)
    
    def saveMusic(self):
        self.filesave = filedialog.asksaveasfile(initialdir='/', title="Save as ", filetype=[("Audio file", "*.mp3"), ("All file", "*.*")], defaultextension=".mp3")
        Music.write_mp3(self.filesave.name, self.fs1, self.YC, normalized=True) 

        
    def playOriginalMusic(self):
        self.ti = self.initial.get()
        self.tf = self.final.get()
        Y = self.data[int(float(self.ti)*44100): int(float(self.tf)*44100), 1];
        Y /= np.max(np.abs(Y))
        sd.play(Y, self.fs1)
        
    
    def applyChorus(self):
        self.ti = self.initial.get()
        self.tf = self.final.get()
        self.Kval = self.K.get()
        self.Tval = self.T.get()
        mus = 0
        for i in range(int(self.Kval)):
            mus += self.data[int((float(self.ti)*44100))+int(float(self.Tval)*i*44.1): int((float(self.tf)*44100))+int(float(self.Tval)*i*44.1), 1];
        mus /= np.max(np.abs(mus))
        self.YC = mus
        sd.play(self.YC, self.fs1)
        
window1 = ChorusGUI()
window1.show()


In [5]:
class EffectGUI:
    def __init__(self):
        self.top = tk.Tk()
        self.top.geometry("700x400") 
        self.top.title("3D Effect GUI")
        self.val = 0
        
    def show(self):        
        Xoffset = 50;
        Yoffset = 0;
        
        img2 = ImageTk.PhotoImage(Image.open("2.jpg"))
        panel2 = tk.Label(self.top, image = img2)
        panel2.place(x=0, y=212)
        panel2.image = img2
        
        
        self.slider = tk.Scale(self.top, from_=-1, to=1, orient=tk.HORIZONTAL, resolution=0.1, command=self.getNew)
        self.slider.place(x=270+Xoffset, y=30+Yoffset)  
        
        tk.Label(self.top, text="Enter time period [s]: ").place(x=10+Xoffset, y=40+Yoffset)
        self.initial = tk.Entry(self.top, width=7)
        self.initial.place(x=127+Xoffset, y=40+Yoffset)
        tk.Label(self.top, text=":").place(x=175+Xoffset, y=40+Yoffset)
        self.final = tk.Entry(self.top, width=7)
        self.final.place(x=185+Xoffset, y=40+Yoffset)
        
        tk.Label(self.top, text='L').place(x=270+Xoffset, y = 10+Yoffset)

        tk.Label(self.top, text='R').place(x=360+Xoffset, y = 10+Yoffset)
        
        
        tk.Button(self.top, text="Play", command=self.Play).place(x=40+Xoffset, y=110+Yoffset)
        
        tk.Button(self.top, text="Stop", command=self.Stop).place(x=80+Xoffset, y=110+Yoffset)


        tk.Button(self.top, text="Select a music", command=self.BrowseMusic).place(x=40+Xoffset, y=140+Yoffset)

        tk.Button(self.top, text="Save music", command=self.saveMusic).place(x=130+Xoffset, y=140+Yoffset)
        
        self.top.mainloop()
    
    def BrowseMusic(self):
        self.filename = filedialog.askopenfilename(initialdir='/', title="Select a music", filetype=[("mp3", "*.mp3")])
        self.fs1, self.data = Music.read_mp3(self.filename, True)
    
    def saveMusic(self):
        self.filesave = filedialog.asksaveasfile(initialdir='/', title="Save as ", filetype=[("Audio file", "*.mp3"), ("All file", "*.*")], defaultextension=".mp3")
        Music.write_mp3(self.filesave.name, self.fs1, self.Y, normalized=True) 

    
    def getNew(self, event):
        self.val = self.slider.get()
        print(self.val)
    
    def Play(self):
        self.r 
        self.Y = np.zeros((self.r-self.l, 2))
        self.Y[:, 0] = np.multiply(self.data[self.l:self.r, 1], 1-self.val)
        self.Y[:, 1] = np.multiply(self.data[self.l:self.r, 0], 1+self.val)
        sd.play(self.Y, self.fs1)
     
    def Stop(self):
        sd.stop()
            
        
one = EffectGUI()
one.show()
    
